# Analysis

## Prepare dataset

In [1]:
from pandas import read_csv

wikidata_lexemes_loc = './datasets/wikidata-english-lexemes.csv'
concreteness_ratings_loc = './datasets/concreteness-ratings-brysbaert-et-al.csv'

wikidata_lexemes_df = read_csv(wikidata_lexemes_loc)
concreteness_ratings_df = read_csv(concreteness_ratings_loc)

print(f'Loaded Wikidata Lexemes (Rows: {wikidata_lexemes_df.shape[0]}).\nSchema:')
wikidata_lexemes_df.info()
print(f'\nLoaded Concreteness Ratings (Rows: {concreteness_ratings_df.shape[0]}).\nSchema:')
concreteness_ratings_df.info()

Loaded Wikidata Lexemes (Rows: 17524).
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17524 entries, 0 to 17523
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lexemeId  17524 non-null  object
 1   lemma     17522 non-null  object
 2   image     8024 non-null   object
dtypes: object(3)
memory usage: 410.8+ KB

Loaded Concreteness Ratings (Rows: 39954).
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39954 entries, 0 to 39953
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Word           39953 non-null  object 
 1   Bigram         39954 non-null  int64  
 2   Conc.M         39954 non-null  float64
 3   Conc.SD        39954 non-null  float64
 4   Unknown        39954 non-null  int64  
 5   Total          39954 non-null  int64  
 6   Percent_known  39954 non-null  float64
 7   SUBTLEX        39954 non-null  int64  
 8   Dom_Pos 

In [ ]:
import pandas as pd

# Join the two datasets
def str_merge(part_string_df, full_string_df, merge_column_1, merge_column_2):
    merge_column_lower = 'merge_column_lower'
    
    part_string_df[merge_column_lower] = part_string_df[merge_column_1].str.lower()
    full_string_df[merge_column_lower] = full_string_df[merge_column_2].str.lower()
    
    pat = '|'.join(r'{}'.format(x) for x in part_string_df[merge_column_lower])
    full_string_df['matched_column'] = full_string_df[merge_column_lower].str.extract(r'\b('+ pat + r')\b', expand=True)
    
    merged_dfs = pd.merge(part_string_df, full_string_df, left_on= merge_column_lower, right_on='matched_column').drop([merge_column_lower + '_x',merge_column_lower + '_y','matched_column'],axis=1)
    return merged_dfs

merged_df = str_merge(concreteness_ratings_df, wikidata_lexemes_df, 'Word', 'lemma')[['lemma', 'Word', 'image', 'Conc.M']]
# Tidy up the columns
merged_df.columns = ['lemma', 'word', 'image', 'concreteness']
merged_df = merged_df[merged_df['word'].notnull()]
merged_df['has_image'] = pd.notnull(merged_df['image'])

merged_df = merged_df[['has_image', 'concreteness']]

print('Dataset after merge:')
print(merged_df.shape)
print(merged_df)

has_image_df = merged_df[merged_df['has_image']].reset_index()
no_image_df = merged_df[~merged_df['has_image']].reset_index()

random_state = 50
sample_size = 1000
has_image_sample_df = has_image_df.sample(n=sample_size, random_state=random_state).reset_index()[['concreteness']]
no_image_sample_df = no_image_df.sample(n=sample_size, random_state=random_state).reset_index()[['concreteness']]

print('Entities that have an image:')
print(has_image_df.shape)
print(has_image_sample_df)

print('Entities that do not have an image:')
print(no_image_df.shape)
print(no_image_sample_df)


## Significance Testing

In [ ]:
import statsmodels.stats.weightstats as ws

# Sample size is > 30, and we can know the standard deviation.
# Therefore, we will use a Z-Test to test for significance.
col1 = ws.DescrStatsW(has_image_sample_df['concreteness'])
col2 = ws.DescrStatsW(no_image_sample_df['concreteness'])
cm_obj = ws.CompareMeans(col1, col2)
zstat, z_pval = cm_obj.ztest_ind(usevar='unequal')
print(f'Z-Test Results:\nstatistic: {zstat}\np-value: {z_pval}')

if z_pval <= 0.05:
    print('Z Value is less than 0.05 - Rejecting the null hypothesis and accepting the alternative hypothesis.')
else:
    print('Z Value is greater than 0.05 - Rejecting the alternative hypothesis and accepting the null hypothesis.')

## Visualisations 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = ['Lexemes with an image', 'Lexemes without an image']

# 1. Plot the total number of Wikidata nouns that have an image/don't in a pie chart
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%\n(n={v:d})'.format(p=pct,v=val)
    return my_autopct

pie_data = np.array([has_image_df.shape[0], no_image_df.shape[0]])
pie_explode = [0, 0.1]

plt.figure(1)
plt.pie(pie_data, labels = labels, startangle = 90, explode = pie_explode, shadow = True, autopct=make_autopct(pie_data))
plt.title('Wikidata noun lexemes overview')
plt.show()

# 2. The mean of each dataset
plt.figure(2)
data = [has_image_df['concreteness'].mean(), no_image_df['concreteness'].mean()]
plt.bar(labels, data)
plt.title('The mean concreteness scores of each population')
plt.xlabel('Population')
plt.ylabel('Mean Concreteness Score')
plt.show()

# 3. Plot the distribution of concreteness ratings for both datasets
data1 = np.array(has_image_df.round(0)['concreteness'].value_counts().sort_index())
data2 = np.array(no_image_df.round(0)['concreteness'].value_counts().sort_index())
width = 0.3


plt.figure(3)
plt.bar([0.85, 1.85, 2.85, 3.85, 4.85], data1, width = width)
plt.bar([1.15, 2.15, 3.15, 4.15, 5.15], data2, width=width)
plt.title('Concreteness rating distribution in relation to whether a noun has an image or not')
plt.xlabel('Concreteness Rating (Rounded)')
plt.ylabel('Count')
plt.legend(labels)
plt.show()
